In [4]:
def naver_kin(search,start_date,end_date,page_num=1):
    import time
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    import pandas as pd

    options=Options()
    options.add_experimental_option('detach',True)
    options.add_argument('--start-maximized')
    service=Service(ChromeDriverManager().install())

    driver=webdriver.Chrome(service=service, options=options)
    
    # search_date_length = (dt.datetime.strptime(end_date,"%Y.%m.%d")-dt.datetime.strptime(start_date,"%Y.%m.%d")).days
    # search = '부동산'
    # start_date = '2024.02.01'
    # end_date = '2024.02.08'
    res = [{'질문':""}]
    for page in range(1, page_num+1):
        url = f'https://kin.naver.com/search/list.naver?query={search}&section=qna&period={start_date}.|{end_date}.&sort=none&page={page}'

        driver.get(url)
        time.sleep(1)
        
        for list_num in range(1,11):
            try:
                tmp={}
                driver.find_element(By.XPATH,f'//*[@id="s_content"]/div[3]/ul/li[{list_num}]/dl/dt/a').click()
                time.sleep(1)
                driver.switch_to.window(driver.window_handles[-1])
                tmp['질문'] = driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[1]/div[1]').text.replace('질문\n','')
                tmp['상세질문'] = driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[1]/div[3]').text.replace('\n','')
                tmp['답변개수'] = driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[7]/div[1]/div/div/div[1]/span').text
                tmp['답변'] = [i.text for i in driver.find_elements(By.CLASS_NAME,'se-component-content')]
                driver.close()
                driver.switch_to.window(driver.window_handles[-1])
                if tmp['질문'] == res[-1]['질문']:
                    break
                res.append(tmp)
            except:
                print(f"{page}페이지 {list_num}번째 자료 오류로인하여 수집되지 않았습니다.")
    driver.quit()
    res = res[1:]
    df = pd.DataFrame(res)
    print(f'데이터 {df.shape} 처리되었습니다.')
    df.to_csv(f'네이버지식인_{search}_{start_date}_{end_date}_{page_num}.csv')
    return df


In [2]:
df = naver_kin('부동산','2024.10.01','2024.10.04',20)
df

,질문,상세질문,답변개수,답변
0,부동산 상속 관련 문의드려요,"외할아버지가 돌아가셨는데엄마,삼촌,이모 셋이서 합의를 보고 엄마한테 부동산을 상속하...",3,[안녕하세요. 서울지방변호사회 소속으로 지식iN 법률상담을 진행하고 있는 김솔애 변...
1,제주 부동산 특조법,제주에 땅이 작은 거 (170평)있는데ᆢ안에 미등기 묘지 지목(50평)만 있어서 (...,2,"[안녕하세요. 로시컴-네이버 지식iN 상담변호사 임선준 입니다., ⚖️철저하고 꼼꼼..."
2,아파트 가계약하고 집주인이 더높은 가격으로 부동산에...,아파트 가계약하고 집주인이 더높은 가격으로 부동산에 내놓았습니다이 상황만으로 계약을...,5,[1. 그런 경우 계약금의 배액을 배상 받고 해약을 하면 될 일입니다.\n주인이 할...
3,부동산 가계약금 반환,집을 보고 마음에들어서 가계약금 30만원을 집주인에게 입금했습니다그런데 그날 부동산...,3,[가계약금의 의미는\n상황에 따라 다를 수 있어서\n생각보다 간단한 문제는 아닙니다...
4,전세사기 부동산강제경매,안녕하세요2023년에 전세사기 당한것을 알고임차권 등록 진행해서 허그보증보험을 통해...,2,"[귀하는 별지목록기재 부동산에 대한 주택임차권자인바, 위 부동산에 대하여 채권자로부..."
...,...,...,...,...
195,집 매매 계약했는데~,경기도에 집을 매매하게 되었습니다부동산에서 계약서를 쓰면서 집 가격이1억8천5백만원...,2,[경기도에 집을 매매하게 되었습니다\n부동산에서 계약서를 쓰면서 집 가격이\n1억 ...
196,전세 보증금 받는 법,"빌라 만기계약 6개월전 전화로 이사하겠다 말하고집주인은 안가면안되냐, 꼭 가야하냐 ...",4,[제가 부린이라 잘 몰라요..\n지금 신혼부부디딤돌전세대출인데 2년연장해서 4년째 ...
197,집 매매 하려고 하는데 중요한게 있을까요?,24살이구 부동산용어도 모르고 절차도 모릅니다.오늘 가계약 걸러 가는데 아버지는 융...,1,"[어떤 지역이고, 주택 빌라 아파트인지, 비규제 지역인지 규제지역인지,\n다 따져 ..."
198,"공증 어길 시, 질문드립니다.","안녕하세요,저희 업체가 건설사를 일을 하다보니 자금 흐름이 원활하지 않습니다.그러다...",1,[안녕하세요. 서울지방변호사회 소속으로 지식iN 법률상담을 진행하고 있는 김솔애 변...


In [2]:
import tkinter as tk
from tkcalendar import DateEntry

win = tk.Tk()

win.geometry('600x900')
win.title('네이버 지식인 검색')
win.option_add('*Font','맑은고딕 20')

f1 = tk.Frame(win,width=1500,height=100,relief=tk.SUNKEN,bd=4,bg='light steel blue')
f1.pack(side=tk.TOP)
f2 = tk.Frame(win,width=1500,height=550,relief=tk.SUNKEN,bd=4,bg='white')
f2.pack()
f3 = tk.Frame(win,width=1600,height=100,relief=tk.SUNKEN,bd=4,bg='white')
f3.pack(side=tk.BOTTOM)

# 네이버 지식인 로고
img = tk.PhotoImage(file='./img/네이버 지식인 로고.png', master=f1).subsample(1)
lab_d = tk.Label(f1, image=img)
lab_d.grid()

# 검색창
lab1 = tk.Label(f2, text='검색어')
lab1.grid(row=0,column=0)

ent1 = tk.Entry(f2)
ent1.grid(row=0,column=1)

# 시작날짜
lab2 = tk.Label(f2, text='시작날짜')
lab2.grid(row=1,column=0)

s_cal = DateEntry(f2, locale='ko', date_pattern='Y.mm.dd')
s_cal.grid(row=2,column=0)

# 끝날짜
lab2 = tk.Label(f2, text='끝날짜')
lab2.grid(row=1,column=1)

e_cal = DateEntry(f2, locale='ko', date_pattern='Y.mm.dd')
e_cal.grid(row=2,column=1)

# 페이지수
lab2 = tk.Label(f2, text='페이지 수(1~100)')
lab2.grid(row=3,column=0)

ent2 = tk.Entry(f2)
ent2.grid(row=3,column=1)

btn = tk.Button(f2, text='검색')
btn.grid(row=4,column=0,columnspan=2)

win.mainloop()